<a href="https://colab.research.google.com/github/tomioka52q/univercity/blob/main/tasks/%22%D0%94%D0%BE%D0%BF%D0%BE%D0%BB%D0%BD%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%E2%84%962_2_ipynb%22_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [ ]:
import time

def measure(func):
    def wrapper(n):
        start = time.time()
        result = func(n)
        end = time.time()
        print(f"{func.__name__} выполнилась за {end - start:.4f} секунд")
        return result
    return wrapper

@measure
def even_with_append(n):
    numbers = []
    for i in range(n + 1):
        if i % 2 == 0:
            numbers.append(i)
    return numbers

@measure
def even_with_comprehension(n):
    return [i for i in range(n + 1) if i % 2 == 0]

N = 10000000
list1 = even_with_append(N)
list2 = even_with_comprehension(N)

even_with_append выполнилась за 0.6685 секунд
even_with_comprehension выполнилась за 0.6289 секунд


---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [ ]:
def fibonacci_cache(func):
    cache = [0, 1]

    def wrapper(n):
        if n < len(cache):
            return cache[:n + 1]

        current = len(cache)
        while current <= n:
            next_fib = cache[current - 1] + cache[current - 2]
            cache.append(next_fib)
            current += 1

        return cache[:n + 1]

    return wrapper
@fibonacci_cache
def fibonacci(n):

    pass

print(f"{fibonacci(5)}")
print(f"{fibonacci(3)}")
print(f"{fibonacci(7)}")
print(f"{fibonacci(2)}")
print(f"{fibonacci(10)}")

[0, 1, 1, 2, 3, 5]
[0, 1, 1, 2]
[0, 1, 1, 2, 3, 5, 8, 13]
[0, 1, 1]
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [ ]:
import time

def measure(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Время: {end - start:.6f} секунд")
        return result
    return wrapper
def cache_decorator(func):
    cache = [0, 1]

    def wrapper(n):
        print(f"Кэш для n={n}, размер: {len(cache)}")
        if n < len(cache):
            print(f"из кэша!")
            return cache[:n+1]
        else:
            print(f"Вычисляем с {len(cache)} до {n}")
            current = len(cache)
            while current <= n:
                cache.append(cache[current-1] + cache[current-2])
                current += 1
            return cache[:n+1]
    return wrapper

@measure
@cache_decorator
def fib(n):
    return f"Результат для F({n})"
fib(5)
fib(3)
fib(8)


Кэш для n=5, размер кэша: 2
Вычисляем с 2 до 5
Время: 0.000083 секунд
Кэш для n=3, размер кэша: 6
Берем из кэша!
Время: 0.000016 секунд
Кэш для n=8, размер кэша: 6
Вычисляем с 6 до 8
Время: 0.000017 секунд


[0, 1, 1, 2, 3, 5, 8, 13, 21]

---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [ ]:
def make_multiplier(n):
    def multiplier(x):
        return x * n
    return multiplier

times3 = make_multiplier(3)
print(times3(5))


15


---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [ ]:
def rounder(n):
    def round_number(x):
        return round(x, n)
    return round_number

round_to_2 = rounder(2)
print(round_to_2(3.1415926))


3.14


---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [2]:
import time

def time_threshold(threshold=0.5):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = end_time - start_time

            if execution_time > threshold:
                print(f"Функция {func.__name__} выполнялась {execution_time:.4f} секунд (порог: {threshold} сек)")

            return result
        return wrapper
    return decorator

@time_threshold(threshold=0.5)
def long_running_function():
    time.sleep(0.7)
    return "Готово!"

if __name__ == "__main__":

    print("ДОЛЬШЕ")
    result1 = long_running_function()
    print(f"{result1}")

ДОЛЬШЕ
Функция long_running_function выполнялась 0.7002 секунд (порог: 0.5 сек)
Готово!


---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [ ]:
def string_generator(char, times):
    for i in range(1, times + 1):
        yield char * i
for s in string_generator('*', 5):
    print(s)

*
**
***
****
*****


---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [ ]:
def infinite_sequence():
    num = 1
    while True:
        yield num
        num += 1

gen = infinite_sequence()
for _ in range(5):
    print(next(gen))

1
2
3
4
5


---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [ ]:
def combined_lists(lst1, lst2):
    for i in range(min(len(lst1), len(lst2))):
        yield lst1[i]
        yield lst2[i]

for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)

1
a
2
b
3
c


---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [ ]:
def reverse_string(s):
    for i in range(len(s) - 1, -1, -1):
        yield s[i]

for char in reverse_string('hello'):
    print(char)

o
l
l
e
h


---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [ ]:
def powers_of_two(n):
    for i in range(n + 1):
        yield 2 ** i

for num in powers_of_two(5):
    print(num)

1
2
4
8
16
32


---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [ ]:
def number_extractor(s):
    number = ''
    for char in s:
        if char.isdigit():
            number += char
        elif number:
            yield int(number)
            number = ''
    if number:
        yield int(number)

for num in number_extractor('abc123def45gh6'):
    print(num)

123
45
6


---